# PUI2016 Final 

## DATA:

I prepared datasets that are partially processed in advance. You can work with those, or gather and process the data by yourself, or begin working with the preprocessed data and then go back to get and wrangle the original data to obtain the dataset I gave you (that is what I would do). Generally each step outlined in the instructions earns you a point, or a fraction of a point for being done partially. See [PUI2016_RULES](https://github.com/fedhere/PUI2016_final/blob/master/PUI2016_RULES.ipynb) for more details.
     
  

 ## Geospatial files: 
 you will need to join the census dataframes with GeoDataFrames which contain the shapes of the census tracts, for the second part, and zipcodes, for the first part.
 
 you are going to need: 
 
- a census tract shape file : you have used on in the [geospatial lab]( https://github.com/fedhere/PUI2016_fb55/tree/master/Lab9_SRK325) and homework, and I uploaded one on my CartoDB account (see below). Any reproducible way to upload one is ok.
- a zipcode shapefile : you have used on in [HW11](https://github.com/fedhere/PUI2016_fb55/blob/master/HW11_fb55/HW11_Assignment2_solution.ipynb) 
- a smaller resloution contour file may also be useful, such as [this one](http://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/nybb_16d.zip), but it is not necessary 
 
I uploaded the census tract geojson on my cartoDB as **table_2010_census_tracts** and you can use the function defined below with the appropriate call for geojson and reading in the file appropriately


    datastream = queryCartoDB(query, format='GeoJSON')
    datageo  = json.loads(datastream.read())
    data = gpd.GeoDataFrame.from_features(datageo['features'])
    


## Busnisses Census Data
In the first part you need the number of businesses per zipcode. 
You used this very dataset in a homework [HW11](https://github.com/fedhere/PUI2016_fb55/blob/master/HW11_fb55/HW11_Assignment2_solution.ipynb). If you need further help look [here](https://docs.google.com/document/d/1KtJapvlkU8l8-mjxOs8K8PNxmK5xxyNIUyvimFiRDjA/edit?usp=sharing)
Use that homework to point you to the lines of code you need to download the files (you only need the 2000 and 2010 file) and how to read them. Merge them to the zipcode GeoDataFrame just like you did then!

 ##  Census data for demographics: 
 
You have to extract the ***total population, median age, and percentage of white population*** from Census data for the 2000 and 2010 ***decennial census data***. You can find the relevant dataset on American Fact Finder using the advanced search https://factfinder.census.gov/faces/nav/jsf/pages/searchresults.xhtml?refresh=t. If you cannot download the data from within the notebook (its hard) you must clearly state all the teps you take to obtain the dataset and provide the relevant URLs when possible. 
The ***income data*** is stored in different tables also available on American Fact Finder.  You want the median imcome per census tract for 2000 and 2010. You can use the ***annual estimate*** (you have to for 2000 and for consistency you should for 2010 as well). Same rules apply here as they did in the search for population, age, and diversity. 
 
**Alternatively** (no points lost) you can download the data with an SQL query from public data in cartoDB: I provide a function for that purpose below. Run the SQL query with the databse name and retrieve the columns you want (as you did in HW12, and Lab10_hv). 
 
 We define a helper function queryCartoDB wraps around the SQL API of CartoDB. We can pass it a public data source (in the form of 'https://{ACCOUNT}.cartodb.com/api/v2/sql', where ACCOUNT is the owner of the data) and an SQL statement to query the data in various formats including CSV, JSON, and GeoJSON. The result returned from this function is a file-like object, which you can pass to a file readers such as Pandas' read_csv() or the JSON parser. In this final exam, all SQL-based data sets have been setup to be publicly available from the account fb55, so, you don't have to use your accounts.  The SQL contains 4 sources - one for demographic info and one for income info for each year.

- Census 2000: DEC_00_SF1_DP1
- Census 2010: DEC_10_SF1_SF1DP1
- Income Census 2000: DEC_00_SF3_DP3_with_ann.csv
- Income Census 2010: ACS_10_5YR_S1901_with_ann.csv

From the decennial census datasets you should end up with the columns corresponding to 
- The Geo identifiers 
- Total population (Number)
- White population (Percentage)
- Median age

From the income census datasets you should end up with the columns corresponding to 
- The Geo identifiers 
- The median household income (Number)

You can consult the metadata files to find out which columns give you the relevant data and include them in your SQL query . The files are uploaded uploaded in [PUI12016data](http://cosmo.nyu.edu/~fb55/PUI2016/data/) : they have the same name as the corresponding file in the SQL CartoDB database, but the extension is \_metadata.csv, and the capitals will be lower case letters in the cartoDB names (cartoDB changes them upon upoading, but the SQL query is insensitive to that).

In the next cell, I'm going to define the helper function. You can copy and paste this code in your notebook. This is the Python2 version, but in [Lab10_hv](https://github.com/fedhere/PUI2016_fb55/tree/master/Lab11_hv) you also had the Python3 verison

In [1]:
%matplotlib inline

import ast
import json
import urllib
import urllib2
from cStringIO import StringIO
import pandas as pd
import geopandas as gpd


SQL_SOURCE = 'https://fb55.cartodb.com/api/v2/sql'

def queryCartoDB(query, format='CSV', source=SQL_SOURCE):
    """queries data from CartoDB SQL database
    Arguments:
    query: a string like 
            SELECT colum_name_1, column_name_2 FROM database_name
    format: the file format (e.g. CSV, GeoJson)
    source: the sql database url link
    Return:
        database as a string to be read in with the appropriate function
    """
    data = urllib.urlencode({'format': format, 'q': query})
    try:
        response = urllib2.urlopen(source, data)
    except urllib2.HTTPError, e:
        raise ValueError('\n'.join(ast.literal_eval(e.readline())['error']))
    except Exception:
        raise
    return StringIO(response.read())

# call as:
#datastream = queryCartoDB(query, format='CSV')
#data       = pd.read_csv(database_name)



## Census data shortcut
you can get the census data that I wrangled for you from these URLs

[census2000](http://cosmo.nyu.edu/~fb55/PUI2016/data/census00_final.json) : http://cosmo.nyu.edu/~fb55/PUI2016/data/census00_final.json

[census2010](http://cosmo.nyu.edu/~fb55/PUI2016/data/census10_final.json) : http://cosmo.nyu.edu/~fb55/PUI2016/data/census10_final.json            

**You will need to make spatial joints to merge files by ZIP code. Remember that the columns you merge on have to have the same values, and the same types! (both integers, or both strings) **